<h1>Road to Automation!</h1>

In [ ]:
pip --version

In [ ]:
pip install awscli --upgrade --user

In [ ]:
%env AWS_ACCESS_KEY_ID=
%env AWS_SECRET_ACCESS_KEY=
%env AWS_DEFAULT_REGION=us-west-2
%env AWS_DEFAULT_OUTPUT=json

In [ ]:
!aws iam list-policies

In [ ]:
#List all VPCs
!aws ec2 describe-vpcs

In [ ]:
#List only VPC named (tag) "Lab VPC"
!aws ec2 describe-vpcs --filters Name=tag:Name,Values="Lab VPC" --query "Vpcs[*].VpcId"

In [ ]:
#Save VPC ID into a file (for next steps) an check (cat) for valid content (Vpc ID)
!aws ec2 describe-vpcs --filters Name=tag:Name,Values="Lab VPC" --query "Vpcs[*].VpcId" --output text > VpcID
!cat VpcID

In [ ]:
#List security groups
!aws ec2 describe-security-groups --filters "Name=vpc-id,Values=$(cat VpcID)" --query "SecurityGroups[*].{Name:GroupName,ID:GroupId}"

In [ ]:
#Taken from above excersise
%env ID_SECURITY_GROUP=sg-0f6dc91779f24d5c3
#Show Security Group configuration
!aws ec2 describe-security-groups --group-ids $ID_SECURITY_GROUP

In [ ]:
#Add FW rule for port 80
!aws ec2 authorize-security-group-ingress --group-id $ID_SECURITY_GROUP --protocol tcp --port 80 --cidr 0.0.0.0/0

In [ ]:
#Show Security Group configuration (again...to check rule above added)
!aws ec2 describe-security-groups --group-ids $ID_SECURITY_GROUP

In [ ]:
#Show VPC Subnets...
!aws ec2 describe-subnets --filters "Name=vpc-id,Values=$(cat VpcID)"

In [ ]:
#Save Subnet ID into a file (for next steps) and check (cat) for valid content (Vpc ID)
!aws ec2 describe-subnets --filters Name=tag:Name,Values="Public Subnet 1"  --query "Subnets[*].SubnetId" --output text >SubnetID
!cat SubnetID

In [ ]:
!aws ec2 create-key-pair --key-name my-key-pair --query "KeyMaterial" --output text > my-key-pair.pem
!chmod 400 my-key-pair.pem

<b>EBS mapping file creation</b>

In [ ]:
%%writefile mapping.json
[
    {
        "DeviceName": "/dev/sda1",
        "Ebs": {
            "VolumeSize": 8
        }
    }
]

<b>User data file creation</b>

In [ ]:
%%writefile my_script.txt
#!/bin/bash
yum -y install httpd
systemctl enable httpd
systemctl start httpd
echo '<html><h1>Hello From Your Web Server!</h1></html>' > /var/www/html/index.html

In [ ]:
#EC2 Instance creation
!aws ec2 run-instances --image-id ami-0dc8f589abe99f538 --instance-type t3.micro --key-name my-key-pair --security-group-ids $ID_SECURITY_GROUP --associate-public-ip-address --block-device-mappings file://mapping.json --user-data file://my_script.txt --subnet-id $(cat SubnetID) > instance.json

In [ ]:
!cat instance.json

In [ ]:
#get Instance ID
!cat instance.json | python3 -c "import sys, json; print(json.load(sys.stdin)['Instances'][0]['InstanceId'])" > instanceId
!cat instanceId 

In [ ]:
!aws ec2 describe-instance-status --instance-id $(cat instanceId) | python3 -c "import sys, json; print(json.load(sys.stdin)['InstanceStatuses'][0]['InstanceState']['Name'])"

In [ ]:
!aws ec2 describe-instances --instance-ids $(cat instanceId) --query 'Reservations[*].Instances[*].PublicIpAddress' --output text > instancePublicIp
!cat instancePublicIp

In [ ]:
!echo "http://$(cat instancePublicIp)"